# Clustering the protein dataset

The goal is to cluster the data with several different methods, in order to find the annotations that match the clusters thee best. 

#### TODOs


## Creating the datasets

### Protein datasets

In [1]:
from subpred.dataset import create_dataset, SUBSTRATE_KEYWORDS

df_uniprot = create_dataset(
    input_file="../data/raw/uniprot/uniprot_2022_05_evidence1-2_nofragments.tsv",
    # keywords_classes = None,
    # keywords_classes_all = SUBSTRATE_KEYWORDS,
    # keywords_filter = None,
    multi_substrate="keep",
    # outliers=outliers,
    verbose=True,
    # tax_ids_filter=[3702, 9606, 83333, 559292],
    # sequence_clustering=70,
    evidence_code=2,
    invalid_amino_acids="remove_protein",
    # gene_names_only = True,
    # force_update=True,
    # remove_sequence_fragments = True,
    # force_update = False,
    tcdb_substrates_file="../data/raw/tcdb/tcdb_substrates.tsv",
    swissprot_only=False,
)
print(df_uniprot.shape)
df_uniprot.head()


Found pickle, reading...
(1021426, 16)


,gene_names,protein_names,reviewed,protein_existence,sequence,organism_id,go_ids,keyword_ids,keywords,tcdb_id,interpro,tcdb_class,tcdb_substrates,keywords_substrates,keywords_transport_related,keywords_location
Uniprot,,,,,,,,,,,,,,,,
A0A0C5B5G6,MT-RNR1,Mitochondrial-derived peptide MOTS-c (Mitochon...,reviewed,Evidence at protein level,MRWQEMGYIFYPRKLR,9606,GO:0001649; GO:0003677; GO:0005615; GO:0005634...,KW-0238; KW-0496; KW-0539; KW-0892; KW-1185; K...,DNA-binding;Mitochondrion;Nucleus;Osteogenesis...,NaN,NaN,0.0,NaN,,,
A0A1B0GTW7,CIROP LMLN2,Ciliated left-right organizer metallopeptidase...,reviewed,Evidence at protein level,MLLLLLLLLLLPPLVLRVAASRCLHDETQKSVSLLRPPFSQLPSKS...,9606,GO:0004222; GO:0005737; GO:0006508; GO:0007155...,KW-0025; KW-0225; KW-0325; KW-1056; KW-0378; K...,Alternative splicing;Disease variant;Glycoprot...,NaN,IPR001577;,0.0,NaN,,,
A0JNW5,BLTP3B KIAA0701 SHIP164 UHRF1BP1L,Bridge-like lipid transfer protein family memb...,reviewed,Evidence at protein level,MAGIIKKQILKHLSRFTKNLSPDKINLSTLKGEGELKNLELDEEVL...,9606,GO:0005769; GO:0005829; GO:0034498; GO:0042803...,KW-0025; KW-0175; KW-0963; KW-0967; KW-0597; K...,Alternative splicing;Coiled coil;Cytoplasm;End...,NaN,IPR026728;IPR026854;,0.0,NaN,,,
A0JP26,POTEB3,POTE ankyrin domain family member B3,reviewed,Evidence at protein level,MVAEVCSMPAASAVKKPFDLRSKMGKWCHHRFPCCRGSGKSNMGTS...,9606,NaN,KW-0025; KW-0040; KW-0175; KW-1185; KW-0677,Alternative splicing;ANK repeat;Coiled coil;Re...,NaN,IPR002110;IPR036770;IPR039497;,0.0,NaN,,,
A0PK11,CLRN2,Clarin-2,reviewed,Evidence at protein level,MPGWFKKAWYGLASLLSFSSFILIIVALVVPHWLSGKILCQTGVDL...,9606,GO:0007605; GO:0032421; GO:0060088; GO:0060171...,KW-1003; KW-0966; KW-0209; KW-0225; KW-0325; K...,Cell membrane;Cell projection;Deafness;Disease...,9.A.46.1.2,IPR026748;,9.A,NaN,,,


In [2]:
df_swissprot = df_uniprot[df_uniprot.reviewed == "reviewed"]
df_swissprot.shape


(146595, 16)

### Annotation datasets

In [3]:
from subpred.go_utils import GeneOntology

go = GeneOntology("../data/raw/ontologies/go.owl")


In [4]:
from subpred.dataset import get_go_df, get_keywords_df, get_tcdb_substrates

df_kw_uniprot = get_keywords_df(df_uniprot)
df_kw_uniprot.head()

,Uniprot,keyword
0,A0A0C5B5G6,DNA-binding
1,A0A0C5B5G6,Mitochondrion
2,A0A0C5B5G6,Nucleus
3,A0A0C5B5G6,Osteogenesis
4,A0A0C5B5G6,Reference proteome


In [5]:
df_kw_uniprot_long = df_kw_uniprot.assign(dataset="keyword").rename(columns={"keyword": "annotation"})[
    ["Uniprot", "dataset", "annotation"]
]
df_kw_uniprot_long.head()

,Uniprot,dataset,annotation
0,A0A0C5B5G6,keyword,DNA-binding
1,A0A0C5B5G6,keyword,Mitochondrion
2,A0A0C5B5G6,keyword,Nucleus
3,A0A0C5B5G6,keyword,Osteogenesis
4,A0A0C5B5G6,keyword,Reference proteome


In [6]:
# TODO annotations are missing because the terms are deprecated
df_go_uniprot = get_go_df(df_uniprot, go)
df_go_uniprot.head()

4735247
4735247


,Uniprot,go_id,go_term
0,A0A0C5B5G6,GO:0001649,osteoblast differentiation
1,A0A0C5B5G6,GO:0003677,DNA binding
2,A0A0C5B5G6,GO:0005615,extracellular space
3,A0A0C5B5G6,GO:0005634,nucleus
4,A0A0C5B5G6,GO:0005739,mitochondrion


In [7]:
# TODO test if there are empty values in go df
df_go_uniprot[df_go_uniprot.go_term == ""]

,Uniprot,go_id,go_term


In [8]:
df_go_uniprot_long = df_go_uniprot.melt(
    id_vars=["Uniprot"],
    value_vars=["go_id", "go_term"],
    var_name="dataset",
    value_name="annotation",
)
df_go_uniprot_long = df_go_uniprot_long[df_go_uniprot_long.dataset == "go_term"]
df_go_uniprot_long.head()

,Uniprot,dataset,annotation
4735247,A0A0C5B5G6,go_term,osteoblast differentiation
4735248,A0A0C5B5G6,go_term,DNA binding
4735249,A0A0C5B5G6,go_term,extracellular space
4735250,A0A0C5B5G6,go_term,nucleus
4735251,A0A0C5B5G6,go_term,mitochondrion


In [9]:
# TODO TCDB classes
df_tcdb_uniprot = get_tcdb_substrates(df_uniprot)
df_tcdb_uniprot.head()

,Uniprot,tcdb_id,chebi_id,chebi_term
0,A6NI61,1.N.2.1.1,CHEBI:25367,molecule
1,A6NKB5,9.A.80.1.1,CHEBI:24636,proton
2,O00168,1.A.27.1.8,CHEBI:22563,anion
3,O00337,2.A.41.2.3,CHEBI:9175,sodium(1+)
4,O00337,2.A.41.2.3,CHEBI:9893,uridine


In [10]:
df_tcdb_uniprot_long = df_tcdb_uniprot.melt(
    id_vars=["Uniprot"],
    value_vars=["tcdb_id", "chebi_id", "chebi_term"],
    var_name="dataset",
    value_name="annotation",
)
df_tcdb_uniprot_long.head()

,Uniprot,dataset,annotation
0,A6NI61,tcdb_id,1.N.2.1.1
1,A6NKB5,tcdb_id,9.A.80.1.1
2,O00168,tcdb_id,1.A.27.1.8
3,O00337,tcdb_id,2.A.41.2.3
4,O00337,tcdb_id,2.A.41.2.3


In [11]:
import pandas as pd

df_uniprot_interpro = (
    df_uniprot.interpro.dropna().str.rstrip(";").str.split(";").explode().str.strip()
)
assert df_uniprot_interpro[~df_uniprot_interpro.str.match("IPR[0-9]{6}")].shape[0] == 0
df_uniprot_interpro = df_uniprot_interpro.reset_index(drop=False)  # [df_uniprot_interpro.isnull()]
df_uniprot_interpro

,Uniprot,interpro
0,A0A1B0GTW7,IPR001577
1,A0JNW5,IPR026728
2,A0JNW5,IPR026854
3,A0JP26,IPR002110
4,A0JP26,IPR036770
...,...,...
4188487,X5MFI4,IPR029044
4188488,X5MI49,IPR008630
4188489,X5MI49,IPR029044
4188490,X5MPI5,IPR008630


In [12]:
df_uniprot_interpro_long = df_uniprot_interpro.assign(dataset="interpro").rename(columns={"interpro": "annotation"})[
    ["Uniprot", "dataset", "annotation"]
]
df_uniprot_interpro_long.head()

,Uniprot,dataset,annotation
0,A0A1B0GTW7,interpro,IPR001577
1,A0JNW5,interpro,IPR026728
2,A0JNW5,interpro,IPR026854
3,A0JP26,interpro,IPR002110
4,A0JP26,interpro,IPR036770


In [13]:
df_annotations = pd.concat(
    [
        df_kw_uniprot_long,
        df_go_uniprot_long,
        df_tcdb_uniprot_long,
        df_uniprot_interpro_long,
    ],
    ignore_index=True,
)
df_annotations.head()


,Uniprot,dataset,annotation
0,A0A0C5B5G6,keyword,DNA-binding
1,A0A0C5B5G6,keyword,Mitochondrion
2,A0A0C5B5G6,keyword,Nucleus
3,A0A0C5B5G6,keyword,Osteogenesis
4,A0A0C5B5G6,keyword,Reference proteome


In [14]:
# TODO ??
# implement function get_labels, that returns all labels for a go term
df_annotations[df_annotations.duplicated()]

,Uniprot,dataset,annotation
5414240,P00519,go_term,signal transduction in response to DNA damage
5496325,Q9HBM1,go_term,kinetochore
5534962,Q6P1K2,go_term,kinetochore
5538705,Q8NBT2,go_term,kinetochore
5542601,Q96IY1,go_term,kinetochore
...,...,...,...
10167603,Q9V7S5,chebi_term,phosphate(3-)
10168287,Q9RQ29,chebi_term,long-chain fatty acid
10168579,O32140,chebi_term,"7,9-dihydro-1H-purine-2,6,8(3H)-trione"
10168711,Q180E3,chebi_term,riboflavin


In [15]:
df_go_uniprot[df_go_uniprot.go_term == ""]

,Uniprot,go_id,go_term
